In [6]:
val sparksession=new SparkSession.Builder().appName("incompletesession").getOrCreate()

sparksession = org.apache.spark.sql.SparkSession@5020a3d6


org.apache.spark.sql.SparkSession@5020a3d6

In [7]:
val movies_csv_path="gs://movielens_dataset_rd/movies.csv" 
val moviesDfFull=sparksession.read.option("header", "true").option("inferSchema", "true").csv(movies_csv_path)

movies_csv_path = gs://movielens_dataset_rd/movies.csv
moviesDfFull = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [8]:
val moviesDf=moviesDfFull.limit(10000)

moviesDf = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [9]:
moviesDf.limit(5).show(truncate=false)

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+



In [10]:
val metadata_json_path="gs://movielens_dataset_rd/metadata.json"
val metadataDf = spark.read
  .option("multiline", "true")  // Use this if your JSON file contains multiline JSON records
  .json(metadata_json_path)     // Path to the JSON file in GCS

// Show the first few records of the DataFrame
metadataDf.show()

+------------+
|missing_year|
+------------+
|        1998|
+------------+



metadata_json_path = gs://movielens_dataset_rd/metadata.json
metadataDf = [missing_year: bigint]


[missing_year: bigint]

In [11]:
metadataDf.columns.toList

List(missing_year)

In [12]:
val moviesDfRdd=moviesDf.rdd

moviesDfRdd = MapPartitionsRDD[42] at rdd at <console>:24


MapPartitionsRDD[42] at rdd at <console>:24

In [13]:
//doing the filter using rdd
val filteredRdd=moviesDfRdd.filter(row=>{
    val title=row.getAs[String]("title")
    if(title.length<=6)false
    val yearValueWithBrackets=title.substring(title.length-6)
    
    yearValueWithBrackets.matches("""^\(\d{4}\)$""")
    
})

filteredRdd = MapPartitionsRDD[43] at filter at <console>:25


MapPartitionsRDD[43] at filter at <console>:25

In [14]:
filteredRdd.take(5).foreach(println)

[1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy]
[2,Jumanji (1995),Adventure|Children|Fantasy]
[3,Grumpier Old Men (1995),Comedy|Romance]
[4,Waiting to Exhale (1995),Comedy|Drama|Romance]
[5,Father of the Bride Part II (1995),Comedy]


In [15]:
//printing original number of records
println(moviesDfRdd.count())

//printing reduced number of records
println(filteredRdd.count())

//printing the number of faulty records

println(s"Number of faulty records is ${moviesDfRdd.count()-filteredRdd.count()}")

10000
9992
Number of faulty records is 8


In [16]:
import org.apache.spark.sql.Row
moviesDfRdd.take(5).foreach(element =>
  element match {
    case row: Row => println(s"Row: ${row}")  // Match Row type
//     case (movieId:Int,title:String,genres:String)=>println(s"Tuple : $movieId ,$title ,$genres")
    case _ => println("Other type")
  }
)

Row: [1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy]
Row: [2,Jumanji (1995),Adventure|Children|Fantasy]
Row: [3,Grumpier Old Men (1995),Comedy|Romance]
Row: [4,Waiting to Exhale (1995),Comedy|Drama|Romance]
Row: [5,Father of the Bride Part II (1995),Comedy]


In [17]:
def hasYear(title:String):Boolean={
//     val title=row.getAs[String]("title")
    if(title.length<=6)false
    val yearValueWithBrackets=title.substring(title.length-6)
    
    yearValueWithBrackets.matches("""^\(\d{4}\)$""")
    
}

hasYear: (title: String)Boolean


In [18]:
val missingYear = metadataDf.first().getAs[Long]("missing_year").toInt

missingYear = 1998


1998

In [20]:
def getYearAndTitle(row:Row):(Int,String)={
    val title=row.getAs[String]("title")
    
    val newTitle=if(hasYear(title))title.substring(0,title.length-6).trim else title.trim
    
    val year:Int=if(hasYear(title))Integer.parseInt(title.substring(title.length-5,title.length-1))else missingYear
    
    (year,newTitle)
}

getYearAndTitle: (row: org.apache.spark.sql.Row)(Int, String)


In [21]:
val correctedRdd=moviesDfRdd.map(row=>{
    
//     if(filterUnstructured(row)){
//         row
//     }
//     else{
        
//         val title=row.getAs[String]("title")
//         val newTitle=s"(${title.trim()})"
//         Row(row.getAs[Integer]("movieId"),newTitle,row.getAs[String]("genres"))
        
//     }
    val movieId=row.getAs[Integer]("movieId")
    val (year,title)=getYearAndTitle(row)
    val genres=row.getAs[String]("genres")
    
    (movieId,title,year,genres)
    
})

correctedRdd = MapPartitionsRDD[47] at map at <console>:26


MapPartitionsRDD[47] at map at <console>:26

In [22]:
correctedRdd.take(5).foreach(println)

(1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy)
(2,Jumanji,1995,Adventure|Children|Fantasy)
(3,Grumpier Old Men,1995,Comedy|Romance)
(4,Waiting to Exhale,1995,Comedy|Drama|Romance)
(5,Father of the Bride Part II,1995,Comedy)


In [25]:
//convert the RDD into a Dataframe
val correctedDf=correctedRdd.toDF("movieId","Title","Release Year","Genres")


correctedDf = [movieId: int, Title: string ... 2 more fields]


[movieId: int, Title: string ... 2 more fields]

In [23]:

val outputDir = s"hdfs:///output/caseStudy3/WithRdds/user-data/correctedMovieData"



outputDir = hdfs:///output/caseStudy3/WithRdds/user-data/correctedMovieData


hdfs:///output/caseStudy3/WithRdds/user-data/correctedMovieData

In [26]:
//writing the file to HDFS in Parquet Format

correctedDf.write.parquet(outputDir)